In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F

# Define MLP architecture
class MLP(nn.Module):
    def __init__(self, channel, num_classes):
        super(MLP, self).__init__()
        self.fc_1 = nn.Linear(28 * 28 * 1 if channel == 1 else 32 * 32 * 3, 128)
        self.fc_2 = nn.Linear(128, 128)
        self.fc_3 = nn.Linear(128, num_classes)

    def forward(self, x):
        out = x.view(x.size(0), -1)
        out = F.relu(self.fc_1(out))
        out = F.relu(self.fc_2(out))
        out = self.fc_3(out)
        return out

# Load MNIST dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Initialize MLP model
model = MLP(channel=1, num_classes=10)  # Set channel to 1 for grayscale images, 3 for RGB

# Define loss function, optimizer, and learning rate scheduler
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
scheduler = CosineAnnealingLR(optimizer, T_max=20, eta_min=0.001)

# Train the model with the original dataset
def train(model, train_loader, criterion, optimizer):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

# Test the model
def test(model, test_loader):
    model.eval()
    correct = 0
    total_flops = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            _, predicted = output.max(1)
            correct += predicted.eq(target).sum().item()

            # Calculate FLOPs (Floating-Point Operations Per Second)
            total_flops += 2 * 128 * (28 * 28 * 1 if model.fc_1.in_features == 28 * 28 * 1 else 32 * 32 * 3)  # Assuming 128 units in fc_1

    accuracy = correct / len(test_loader.dataset)
    flops_per_sec = total_flops / (len(test_loader.dataset) * 1.0)

    return accuracy, flops_per_sec

# Train the model for 20 epochs
for epoch in range(20):
    train(model, train_loader, criterion, optimizer)
    scheduler.step()

# Test the model and report the results
accuracy, flops_per_sec = test(model, test_loader)
print(f"Classification Accuracy: {accuracy * 100:.2f}%")
print(f"FLOPs per Second: {flops_per_sec:.2f}")


100%|██████████| 9912422/9912422 [00:00<00:00, 71362238.04it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 3402114.64it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw




100%|██████████| 1648877/1648877 [00:00<00:00, 20586134.79it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 14722201.52it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



Classification Accuracy: 98.12%
FLOPs per Second: 3151.05
